<a href="https://colab.research.google.com/github/DragoneKTM/Digital-signal-and-image-processing/blob/main/DSIM_image_retrival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import zipfile
from shutil import copyfile
import cv2 as cv
from google.colab.patches import cv2_imshow
from cv2 import CascadeClassifier
from sklearn.model_selection import train_test_split

## Load dataset 

VIPs dataset

In [ ]:
# Link Google Drive account, transfer dataset, and extract files
from google.colab import drive
drive.mount('/content/gdrive')

copyfile('gdrive/My Drive/vip.zip', 'vip.zip')
zip = zipfile.ZipFile('vip.zip')
zip.extractall()
zip.close()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Data loader + face detection

In [ ]:
def face_detection(img):
    # funzione che data un'immagine ritorna solamente il crop della faccia che trova al suo interno
    classifier = CascadeClassifier('/haarcascade_frontalface_default.xml')
    box = classifier.detectMultiScale(img, minNeighbors=30)
    #se non trova nessuna faccia non ritorna niente
    (x,y,w,h) = (0,0,0,0)
    if box is ():
        return None
    #altrimenti ritorna solo la prima faccia
    else:
        (x,y,w,h) = box[0]
        return cv.resize(img[y:y+h, x:x+w, :], (224,224))

In [ ]:
def load_data():
    # funzione che per con le immagini crea il dataset di train e test applicando prima la funzione di face detection
    root = '/content/thumbnails_features_deduped_publish'

    labels = []
    images = []

    
    for folder in sorted(os.listdir(root)):
    #dalla cartella principale apro solo le cartelle (no file .txt etc.)
        if not os.path.isfile(os.listdir(root + "" + folder)):
            for file in sorted(os.listdir(root + "" + folder)):
            # per ogni foto che trovo nelle cartelle vips, opero face detection e salvo il risultato
                if file.endswith('.jpg'):
                    img = cv.imread(root + "/" + folder)
                    img = (face_detection(img))
                    #se il face detector trova un'immagine allora la aggiunge al dataset
                    if img is not None:
                        images.append(img)
                        # lable
                        labels.append(str(folder))

    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=1)
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = load_data()